In [1]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'

In [ ]:
!kaggle datasets download -d cactus3/basicshapes

In [ ]:
!unzip \*.zip && rm *.zip

In [4]:
import pathlib
Dir = 'shapes'
dataDir = pathlib.Path(Dir)
dataDir

PosixPath('shapes')

In [24]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [7]:
circles = list(dataDir.glob("circles/*"))
Image.open(circles[0])

In [8]:
triangles = list(dataDir.glob("triangles/*"))
squares = list(dataDir.glob("squares/*"))

In [9]:
Image.open(str(triangles[2]))


In [10]:
Image.open(str(squares[3]))

In [11]:
img_dic ={
    'circle':circles,
    'square':squares,
    'tringle':triangles
}

label_dic ={
    'circle':0,
    'square':1,
    'tringle':2
}

In [17]:
train =[]
test = []

for key,value in img_dic.items():
  for img in value:
    re = cv2.imread(str(img))
    img = cv2.resize(re,(224,224))
    train.append(img)
    test.append(label_dic[key])

according to the data the imgs are 28*28
but according to the pretrained model doc the model accepts only 224*224 imgs, so resize should have to be done.
model link: https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4



In [27]:
train = np.array(train)
test = np.array(test)
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(train,test, train_size=0.8,random_state = 4)

In [28]:
x_train_scaled = x_train / 255
x_test_scaled = x_test / 255

In [15]:
fmodel = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"
pre_trained = hub.KerasLayer(fmodel,input_shape = (224,224,3), trainable = False )

In [16]:
no_of_shapes = 3
model = tf.keras.Sequential([pre_trained, tf.keras.layers.Dense(no_of_shapes)])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_1 (KerasLayer)  (None, 1280)              2257984   
                                                                 
 dense_1 (Dense)             (None, 3)                 3843      
                                                                 
Total params: 2,261,827
Trainable params: 3,843
Non-trainable params: 2,257,984
_________________________________________________________________


In [21]:
model.compile(
    optimizer = 'adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['acc']
)


In [30]:
model.fit(x_train_scaled,y_train , epochs= 10)

Epoch 1/10
8/8 [==============================] - 1s 89ms/step - loss: 0.3168 - acc: 0.9250
Epoch 2/10
8/8 [==============================] - 1s 78ms/step - loss: 0.1205 - acc: 0.9875
Epoch 3/10
8/8 [==============================] - 1s 78ms/step - loss: 0.0651 - acc: 0.9917
Epoch 4/10
8/8 [==============================] - 1s 77ms/step - loss: 0.0466 - acc: 0.9958
Epoch 5/10
8/8 [==============================] - 1s 78ms/step - loss: 0.0361 - acc: 0.9958
Epoch 6/10
8/8 [==============================] - 1s 78ms/step - loss: 0.0297 - acc: 0.9958
Epoch 7/10
8/8 [==============================] - 1s 78ms/step - loss: 0.0254 - acc: 0.9958
Epoch 8/10
8/8 [==============================] - 1s 77ms/step - loss: 0.0227 - acc: 0.9958
Epoch 9/10
8/8 [==============================] - 1s 76ms/step - loss: 0.0200 - acc: 0.9958
Epoch 10/10
8/8 [==============================] - 1s 77ms/step - loss: 0.0178 - acc: 1.0000


In [31]:
model.evaluate(x_test_scaled,y_test)

2/2 [==============================] - 1s 715ms/step - loss: 0.0230 - acc: 1.0000


[0.022987348958849907, 1.0]

In [32]:
import pickle
pickle.dump(model, open('model.pkl','wb'))

INFO:tensorflow:Assets written to: ram://a1d8994a-116c-412d-a147-89e105ce9724/assets


INFO:tensorflow:Assets written to: ram://a1d8994a-116c-412d-a147-89e105ce9724/assets
